In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install -U bitsandbytes accelerate transformers trl peft

In [1]:
# ✅ 1. Đăng nhập Hugging Face
from huggingface_hub import login
login(new_session=False)  #

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import torch, math


2025-11-05 09:16:43.704007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762334203.912356     110 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762334203.975873     110 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

In [3]:
from datasets import load_dataset

# 1️⃣ Load dataset
dataset = load_dataset("hoangcung165/Metal-Health-Data")

# 2️⃣ (Tùy chọn) Lấy 1000 mẫu đầu để test
train_subset = dataset["train"].shuffle(seed=42).select(range(5000))

# 3️⃣ Hàm tiền xử lý theo format của Llama 3 Instruct
def preprocess_function(example):
    system_prompt = "You are a helpful and empathetic AI assistant supporting users in mental health conversations."

    # Bắt đầu bằng system prompt
    formatted_text = (
        "<|begin_of_text|>"
        "<|start_header_id|>system<|end_header_id|>\n"
        f"{system_prompt}<|eot_id|>"
    )

    # Lặp qua danh sách hội thoại trong mỗi example
    for turn in example["conversations"]:
        role = "user" if turn["from"] == "human" else "assistant"
        content = turn["value"].strip()
        formatted_text += (
            f"<|start_header_id|>{role}<|end_header_id|>\n"
            f"{content}<|eot_id|>"
        )

    # Kết thúc bằng token cuối hội thoại
    formatted_text += "<|end_of_text|>"

    return {"text": formatted_text}

# 4️⃣ Xử lý dataset
print("🔧 Đang xử lý dataset...")
processed_ds = train_subset.map(preprocess_function, remove_columns=["conversations"])

# 5️⃣ Hiển thị mẫu sau khi xử lý
print("✅ Mẫu sau khi xử lý:")
print(processed_ds[0]["text"][:1000])

# 6️⃣ Tách train/test
train_test_split = processed_ds.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

print(f"\nTrain samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")


data.json:   0%|          | 0.00/747M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/99086 [00:00<?, ? examples/s]

🔧 Đang xử lý dataset...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

✅ Mẫu sau khi xử lý:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful and empathetic AI assistant supporting users in mental health conversations.<|eot_id|><|start_header_id|>user<|end_header_id|>
Xin chào Alex, tôi hy vọng hôm nay bạn làm tốt. Tôi chỉ muốn nói chuyện với ai đó về cảm giác gần đây của tôi. Tôi cảm thấy thất vọng với sức khỏe của mình.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Xin chào Charlie! Tôi ở đây để lắng nghe và hỗ trợ bạn. Thật tuyệt vời khi bạn đã liên hệ. Bạn có thể cho tôi biết thêm về những gì đang xảy ra với sức khỏe của bạn khiến bạn thất vọng không?<|eot_id|><|start_header_id|>user<|end_header_id|>
Cảm ơn, Alex. Chà, tôi đang bị đau đầu dai dẳng và năng lượng thấp. Tôi đã thử các biện pháp khác nhau nhưng dường như không có tác dụng gì. Nó khá khó chịu.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Tôi hiểu bạn có thể bực bội đến mức nào khi các biện pháp điều trị thông thường của bạn không mang lại hiệu

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.truncation_side = "left"

model.config.use_cache = False


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,                          # Rank của ma trận low-rank (8 là cân bằng RAM/hiệu quả)
    lora_alpha=16,                # Hệ số scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,            # Dropout chống overfitting
    bias="none",
    task_type="CAUSAL_LM"
)


In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # vì đây là causal LM
)


In [8]:
from trl import SFTConfig

sft_config = SFTConfig(
    output_dir="./trained_model",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    warmup_ratio=0.03,
    optim="paged_adamw_32bit",
    lr_scheduler_type="constant",
    fp16=True,
    group_by_length=True,
    save_strategy="steps",
    logging_steps=50,
    max_length=1024,
    report_to="none",
)


In [9]:
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, 
    data_collator=data_collator,
    peft_config=lora_config,
)

Adding EOS to train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009}.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,1.633000
100,1.365900
150,1.272300
200,1.232200
250,1.183700
300,1.144300
350,1.117400
400,1.104900
450,1.086100
500,1.055400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1000, training_loss=1.1213423881530762, metrics={'train_runtime': 22734.0269, 'train_samples_per_second': 0.176, 'train_steps_per_second': 0.044, 'total_flos': 1.818527602768036e+17, 'train_loss': 1.1213423881530762, 'epoch': 1.0})

In [11]:
trainer.push_to_hub("Upload LoRA fine-tuned adapter")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/letri345/trained_model/commit/4d419b6b78b6512552f171d25d44dcd57753fe48', commit_message='Upload LoRA fine-tuned adapter', commit_description='', oid='4d419b6b78b6512552f171d25d44dcd57753fe48', pr_url=None, repo_url=RepoUrl('https://huggingface.co/letri345/trained_model', endpoint='https://huggingface.co', repo_type='model', repo_id='letri345/trained_model'), pr_revision=None, pr_num=None)